# RAG with Docling and 🦙 LlamaIndex

In [1]:
# requirements for this example:
%pip install -qq docling docling-core python-dotenv llama-index-embeddings-huggingface llama-index-llms-huggingface-api llama-index-vector-stores-milvus

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from tempfile import TemporaryDirectory

from dotenv import load_dotenv
from pydantic import TypeAdapter
from rich.pretty import pprint

load_dotenv()

True

In [3]:
import warnings

warnings.filterwarnings(action="ignore", category=UserWarning, module="pydantic|torch")
warnings.filterwarnings(action="ignore", category=FutureWarning, module="easyocr")

## Setup

### Reader and node parser

Below we set up:
- a `Reader` which will be used to create LlamaIndex documents, and
- a `NodeParser`, which will be used to create LlamaIndex nodes out of the documents

In [4]:
from enum import Enum
from typing import Iterable

from llama_index.core.readers.base import BasePydanticReader
from llama_index.core.schema import Document as LIDocument
from pydantic import BaseModel

from docling.document_converter import DocumentConverter


class DocumentMetadata(BaseModel):
    dl_doc_hash: str


class DoclingPDFReader(BasePydanticReader):
    class ParseType(str, Enum):
        MARKDOWN = "markdown"
        # JSON = "json"

    parse_type: ParseType = ParseType.MARKDOWN

    def lazy_load_data(self, file_path: str | list[str]) -> Iterable[LIDocument]:
        file_paths = file_path if isinstance(file_path, list) else [file_path]
        converter = DocumentConverter()
        for source in file_paths:
            dl_doc = converter.convert_single(source).output
            match self.parse_type:
                case self.ParseType.MARKDOWN:
                    text = dl_doc.export_to_markdown()
                # case self.ParseType.JSON:
                #     text = dl_doc.model_dump_json()
                case _:
                    raise RuntimeError(
                        f"Unexpected parse type encountered: {self.parse_type}"
                    )
            excl_metadata_keys = ["dl_doc_hash"]
            li_doc = LIDocument(
                doc_id=dl_doc.file_info.document_hash,
                text=text,
                excluded_embed_metadata_keys=excl_metadata_keys,
                excluded_llm_metadata_keys=excl_metadata_keys,
            )
            li_doc.metadata = DocumentMetadata(
                dl_doc_hash=dl_doc.file_info.document_hash,
            ).model_dump()
            yield li_doc

In [5]:
from llama_index.core.node_parser import MarkdownNodeParser

reader = DoclingPDFReader(parse_type=DoclingPDFReader.ParseType.MARKDOWN)
node_parser = MarkdownNodeParser()
transformations = [node_parser]

One can include add more transformations, e.g. further chunking based on text size / overlap, as shown below:

In [6]:
# from llama_index.core.node_parser import TokenTextSplitter

# splitter = TokenTextSplitter(
#     chunk_size=1024,
#     chunk_overlap=20,
# )
# transformations.append(splitter)

### Embed model

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

### Vector store

In [8]:
INGEST = True  # whether to ingest from scratch or reuse an existing vector store

In [9]:
from llama_index.vector_stores.milvus import MilvusVectorStore

MILVUS_URL = os.environ.get(
    "MILVUS_URL", f"{(tmp_dir := TemporaryDirectory()).name}/milvus_demo.db"
)
MILVUS_COLL_NAME = os.environ.get("MILVUS_COLL_NAME", "basic_llamaindex_pipeline")
MILVUS_KWARGS = TypeAdapter(dict).validate_json(os.environ.get("MILVUS_KWARGS", "{}"))
vector_store = MilvusVectorStore(
    uri=MILVUS_URL,
    collection_name=MILVUS_COLL_NAME,
    dim=len(embed_model.get_text_embedding("hi")),
    overwrite=INGEST,
    **MILVUS_KWARGS,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
from llama_index.core import StorageContext, VectorStoreIndex

if INGEST:
    # in this case we ingest the data into the vector store
    docs = reader.load_data(
        file_path="https://arxiv.org/pdf/2206.01062",  # DocLayNet paper
    )
    pprint(docs, max_length=1, max_string=50, max_depth=4)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        documents=docs,
        embed_model=embed_model,
        storage_context=storage_context,
        transformations=transformations,
    )
else:
    # in this case we just load the vector store index
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        embed_model=embed_model,
    )

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

[
│   Document(
│   │   id_='5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c84663'+14,
│   │   embedding=None,
│   │   metadata={'dl_doc_hash': '5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c84663'+14},
│   │   excluded_embed_metadata_keys=['dl_doc_hash'],
│   │   excluded_llm_metadata_keys=['dl_doc_hash'],
│   │   relationships={},
│   │   text='## DocLayNet: A Large Human-Annotated Dataset for '+50593,
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   )
]

### LLM

In [11]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

HF_API_KEY = os.environ.get("HF_API_KEY")

llm = HuggingFaceInferenceAPI(
    token=HF_API_KEY,
    model_name="mistralai/Mistral-7B-Instruct-v0.3",
)

## RAG

In [12]:
query_engine = index.as_query_engine(llm=llm)
query_res = query_engine.query("How many pages were human annotated?")
pprint(query_res, max_length=1, max_string=250, max_depth=4)

Response(
│   response='80863 pages were human annotated.',
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='8874a117-d181-4f4f-a30b-0b5604370d77',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='3 THE DOCLAYNET DATASET\n\nDocLayNet contains 80863 PDF pages. Among these, 7059 carry two instances of human annotations, and 1591 carry three. This amounts to 91104 total annotation instances. The annotations provide layout information in the shape o'+5775,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=9089,
│   │   │   │   end_char_idx=15114,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.7367570400238037
│   │   ),
│   │   ... +1
│   ],
│   metadata={
│   │   '8874a117-d181-4f4f-a30b-0b5604370d77': {
│   │   │   'dl_doc_hash': '5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc',
│   │   │   ... +1
│   │   },
│   │   ... +1
│   }
)